## Imports

In [1]:
import os
import pandas as pd
import torch

In [2]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print(f"CUDA is available with {torch.cuda.device_count()} GPU(s).")

    # Iterate through available GPUs and print their details
    for i in range(torch.cuda.device_count()):
        print(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        print(f"  Compute Capability: {props.major}.{props.minor}")
        print(f"  Total Memory: {props.total_memory / 1024**3:.2f} GB")
else:
    print("CUDA is not available. Using CPU.")

# To get the current device PyTorch is configured to use (e.g., for tensors)
current_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nCurrent default device for tensors: {current_device}")

CUDA is available with 1 GPU(s).

GPU 0: NVIDIA GeForce GTX 1650
  Compute Capability: 7.5
  Total Memory: 4.00 GB

Current default device for tensors: cuda


## Data Exploration

In [3]:
data_dir = os.path.join("data", "books","raw")
users_dir = os.path.join(data_dir, "Users.csv")
ratings_dir = os.path.join(data_dir, "Ratings.csv")
books_dir = os.path.join(data_dir, "Books.csv")

In [4]:
df_users = pd.read_csv(users_dir)
df_ratings = pd.read_csv(ratings_dir)
df_books = pd.read_csv(books_dir, low_memory=False)

In [5]:
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
df_users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [7]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [8]:
df_users.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [9]:
df_users["User-ID"].nunique()

278858

In [10]:
df_users.shape

(278858, 3)

In [11]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [12]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [13]:
df_books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271358,271360,271358,271360,271360,271357
unique,271360,242135,102022,118,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,17627,7535,2,2,2


In [14]:
df_books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [15]:
df_books["ISBN"].nunique()

271360

In [17]:
df_books["Book-Author"].unique()

array(['Mark P. O. Morford', 'Richard Bruce Wright', "Carlo D'Este", ...,
       'David Biggs', 'Teri Sloat', 'Christopher  Biffle'],
      shape=(102023,), dtype=object)

In [18]:
df_books["Publisher"].unique()

array(['Oxford University Press', 'HarperFlamingo Canada',
       'HarperPerennial', ..., 'Tempo', 'Life Works Books', 'Connaught'],
      shape=(16808,), dtype=object)

In [20]:
df_books[df_books["Publisher"] == "Oxford University Press"].head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
357,0195086295,What a Wonderful World: A Lifetime of Recordings,Bob Thiele,1995,Oxford University Press,http://images.amazon.com/images/P/0195086295.0...,http://images.amazon.com/images/P/0195086295.0...,http://images.amazon.com/images/P/0195086295.0...
397,0198320264,Julius Caesar (Oxford School Shakespeare),William Shakespeare,2001,Oxford University Press,http://images.amazon.com/images/P/0198320264.0...,http://images.amazon.com/images/P/0198320264.0...,http://images.amazon.com/images/P/0198320264.0...
521,0192815318,Cranford (The World's Classics),Elizabeth Gaskell,1982,Oxford University Press,http://images.amazon.com/images/P/0192815318.0...,http://images.amazon.com/images/P/0192815318.0...,http://images.amazon.com/images/P/0192815318.0...
817,0198604025,How Not to Say What You Mean: A Dictionary of ...,R. W. Holder,2003,Oxford University Press,http://images.amazon.com/images/P/0198604025.0...,http://images.amazon.com/images/P/0198604025.0...,http://images.amazon.com/images/P/0198604025.0...


In [ ]:
df_books.shape

(271360, 8)

In [ ]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [ ]:
df_ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [ ]:
df_ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [ ]:
df_ratings["User-ID"].nunique()

105283

In [ ]:
df_ratings["ISBN"].nunique()

340556

In [ ]:
df_ratings.shape

(1149780, 3)

## Data Cleaning

In [ ]:
df_books_cleaned = df_books.dropna()

In [ ]:
df_books_cleaned.shape

(271353, 8)

In [ ]:
df_ratings_cleaned_user = df_ratings[(df_ratings["User-ID"] >= 1) & (df_ratings["User-ID"] <= 278858)]

In [ ]:
df_ratings_cleaned_user.shape

(1149780, 3)

In [ ]:

valid_isbns = set(df_books_cleaned.ISBN)
df_ratings_cleaned_books = df_ratings_cleaned_user[df_ratings_cleaned_user["ISBN"].isin(valid_isbns)]

In [ ]:
df_ratings_cleaned_books.shape

(1031128, 3)

In [ ]:
df_ratings_cleaned_books.reset_index(inplace=True, drop=True)

In [ ]:
df_ratings_cleaned_books.head(10)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276744,038550120X,7
7,276746,0425115801,0
8,276746,0449006522,0
9,276746,0553561618,0


In [ ]:
df_ratings_cleaned_books["Book-Rating"].value_counts()

Book-Rating
0     647291
8      91803
10     71224
7      66401
9      60776
5      45355
6      31687
4       7617
3       5118
2       2375
1       1481
Name: count, dtype: int64

## Data Preprocessing

In [1]:
import json
import os

author_encoder_dir = os.path.join("data", "books", "processed", "author_encoder.json")
publisher_encoder_dir = os.path.join("data", "books", "processed", "publisher_encoder.json")

In [ ]:
with open(author_encoder_dir) as f:
    author_tokenizer = json.load(f)

In [ ]:
with open(author_encoder_dir, "w") as f:
    json.dump(author_tokenizer, f, indent=2)

In [11]:
with open(publisher_encoder_dir) as f:
    publisher_encoder = json.load(f)

In [12]:
with open(publisher_encoder_dir, "w") as f:
    json.dump(publisher_encoder, f, indent=2)

In [27]:
import os
import torch
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.utils import negative_sampling

In [2]:
data = torch.load(os.path.join("data", "books", "processed", "graph.pt"), weights_only=False)

In [3]:
data.node_items

<bound method HeteroData.node_items of HeteroData(
  user={ x=[278858, 384] },
  book={ x=[271356, 387] },
  (user, rates, book)={
    edge_index=[2, 1031132],
    edge_attr=[1031132, 1],
  },
  (book, rev_rates, user)={
    edge_index=[2, 1031132],
    edge_attr=[1031132, 1],
  }
)>

In [16]:
data["user"].x.size(1)

384

In [3]:
split_transform = RandomLinkSplit(
    num_val=0.2, 
    num_test=0.1, 
    is_undirected=False, 
    add_negative_train_samples=True, 
    neg_sampling_ratio=1.0,
    edge_types=[("user", "rates", "book")],
    rev_edge_types=[("book", "rev_rates", "user")]
    )

In [4]:
train, val, test = split_transform(data)

In [17]:
type(train)

torch_geometric.data.hetero_data.HeteroData

In [18]:
train_loader = LinkNeighborLoader(
    train,                                      # Train data
    edge_label_index=("user", "rates", "book"), # Used for sampling the edges in a batch
    num_neighbors={                             # Neighbour sampling criteria
        ("user", "rates", "book"): [10, 10, 10],
        ("book", "rev_rates", "user"): [10, 10, 10]
    },
    batch_size=4096,                            # Number of edges to select in a batch
    shuffle=True                                # Shuffling edges
)

In [24]:
len(train_loader)

177

In [26]:
for batch in train_loader:
    edge_label_index = batch[("user", "rates", "book")].edge_label_index
    user_idx, book_idx = edge_label_index
    print(len(user_idx), len(book_idx))
    print(edge_label_index)
    print(user_idx, book_idx)
    break

4096 4096
tensor([[ 155, 2467, 2571,  ...,  419,  515,  153],
        [1792,  697, 1607,  ...,  433,  945, 1780]])
tensor([ 155, 2467, 2571,  ...,  419,  515,  153]) tensor([1792,  697, 1607,  ...,  433,  945, 1780])


In [21]:
x_dict = {ntype: data[ntype].x for ntype in data.node_types}

In [22]:
x_dict["user"]

tensor([[ 0.1397, -0.0587,  0.0500,  ..., -0.0511, -0.0218,  0.0576],
        [ 0.1021, -0.0035, -0.0734,  ..., -0.0033,  0.0481,  0.0693],
        [ 0.0464,  0.0170, -0.0566,  ..., -0.0160, -0.0526, -0.0423],
        ...,
        [ 0.0159, -0.0044,  0.0020,  ..., -0.0326,  0.0167,  0.0328],
        [ 0.0228, -0.0321,  0.0312,  ...,  0.0540, -0.0238, -0.0140],
        [ 0.0601, -0.0011,  0.0112,  ..., -0.0253, -0.0271, -0.0007]])

In [12]:
edge_index_dict = {etype: data[etype].edge_index for etype in data.edge_types}

In [13]:
edge_index_dict

{('user',
  'rates',
  'book'): tensor([[276724, 276725, 276726,  ..., 276705, 276708, 276720],
         [  2966, 225812,  11053,  ...,  52540,  15978,  56814]]),
 ('book',
  'rev_rates',
  'user'): tensor([[  2966, 225812,  11053,  ...,  52540,  15978,  56814],
         [276724, 276725, 276726,  ..., 276705, 276708, 276720]])}

In [7]:
from src.model import Recommender

In [8]:
model = Recommender({"user": 384, "book": 387, "hidden": 512})

In [9]:
model

Recommender(
  (user_proj): NodeEmbedding(
    (proj_feats): Linear(in_features=384, out_features=512, bias=True)
  )
  (book_proj): NodeEmbedding(
    (proj_feats): Linear(in_features=387, out_features=512, bias=True)
  )
  (layers): ModuleList(
    (0-2): 3 x ConvLayer(
      (conv): HeteroConv(num_relations=2)
      (act): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
  )
  (user_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (book_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)

In [10]:
model(train)

{'book': tensor([[ 2.1190,  0.6166, -0.6380,  ..., -0.6380, -0.5081, -0.6380],
         [-0.6227,  0.7638, -0.6227,  ..., -0.6227, -0.6227, -0.6227],
         [-0.6336,  2.8947, -0.6336,  ...,  0.8550, -0.1510, -0.6336],
         ...,
         [-0.5957, -0.3224, -0.5957,  ..., -0.5957, -0.5957, -0.5957],
         [-0.3239, -0.6058, -0.6058,  ..., -0.6058, -0.6058, -0.6058],
         [ 0.5065, -0.6060, -0.6060,  ..., -0.6060, -0.6060, -0.6060]],
        grad_fn=<NativeLayerNormBackward0>),
 'user': tensor([[-0.6139,  0.6807, -0.6139,  ..., -0.6139,  1.3009,  0.1775],
         [-0.5627,  0.2070, -0.5627,  ..., -0.5627,  1.2797, -0.5627],
         [-0.5585,  1.1369,  1.4250,  ...,  2.1811,  1.5007,  0.1663],
         ...,
         [-0.6259,  0.9237,  2.5051,  ...,  2.9983,  1.1399, -0.6259],
         [-0.6070,  0.9619,  1.8968,  ...,  2.1691, -0.6070, -0.1727],
         [-0.6221,  0.4788,  1.4704,  ...,  1.7801,  0.9404,  0.0173]],
        grad_fn=<NativeLayerNormBackward0>)}

In [39]:
a = torch.tensor((1, 512))

In [41]:
a.size()

torch.Size([2])

In [50]:
a = 1
print(f"a{a:04}")

a0001


In [2]:
import os
import torch

In [3]:
model = torch.load(os.path.join(os.getcwd(), "models", "recommender_02.pt"))

In [4]:
model

OrderedDict([('user_proj.proj_feats.weight',
              tensor([[-0.0364, -0.0060, -0.0283,  ..., -0.0287,  0.0537,  0.0362],
                      [ 0.0230, -0.0384, -0.0032,  ...,  0.0388, -0.0186, -0.0216],
                      [-0.0007, -0.0397, -0.0305,  ..., -0.0363, -0.0059,  0.0264],
                      ...,
                      [ 0.0099,  0.0363,  0.0057,  ..., -0.0355, -0.0150,  0.0007],
                      [-0.0275,  0.0005,  0.0200,  ...,  0.0385, -0.0073, -0.0129],
                      [-0.0401,  0.0376, -0.0454,  ...,  0.0184, -0.0066,  0.0160]],
                     device='cuda:0')),
             ('user_proj.proj_feats.bias',
              tensor([ 0.0156,  0.0323,  0.0334, -0.0195, -0.0514,  0.0159,  0.0427,  0.0319,
                       0.0159,  0.0031, -0.0306, -0.0080, -0.0360, -0.0215, -0.0268, -0.0366,
                      -0.0225,  0.0001, -0.0199,  0.0547, -0.0375,  0.0051,  0.0039, -0.0235,
                      -0.0455, -0.0344,  0.0101, -0.0545, 